In [1]:
import pandas as pd
import os
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [8]:
path = '/Users/jacar/OneDrive/Documentos/Maestria/Ciencia de datos aplicada/Talleres/Taller-2/GroceryStoreDataset-master/dataset/train' 
path_test = '/Users/jacar/OneDrive/Documentos/Maestria/Ciencia de datos aplicada/Talleres/Taller-2/GroceryStoreDataset-master/dataset/test' 

In [7]:
def imagen (i):
    imagen = Image.open(i)
    imagen_rgb = imagen.convert('RGB')
    imagen_rgb = imagen_rgb.resize((348, 348))
    imagen_array = np.array(imagen_rgb)
    imagen_a_planar = imagen_array.flatten()
    x = imagen_a_planar / 255.0
    return x

In [6]:
def imagen_90 (i):
    imagen = Image.open(i)
    imagen_rgb = imagen.convert('RGB')
    imagen_rgb = imagen_rgb.resize((348, 348))
    imagen_rotada = imagen_rgb.rotate(90)
    imagen_array = np.array(imagen_rotada)
    imagen_a_planar = imagen_array.flatten()
    x = imagen_a_planar / 255.0
    return x

In [9]:
def imagen_hor (i):
    imagen = Image.open(i)
    imagen_rgb = imagen.convert('RGB')
    imagen_rgb = imagen_rgb.resize((348, 348))
    img_horizontal = imagen_rgb.transpose(Image.FLIP_LEFT_RIGHT)
    imagen_array = np.array(img_horizontal)
    imagen_a_planar = imagen_array.flatten()
    x = imagen_a_planar / 255.0
    return x

In [10]:
def imagen_ver (i):
    imagen = Image.open(i)
    imagen_rgb = imagen.convert('RGB')
    imagen_rgb = imagen_rgb.resize((348, 348))
    img_vertical = imagen_rgb.transpose(Image.FLIP_TOP_BOTTOM)
    imagen_array = np.array(img_vertical)
    imagen_a_planar = imagen_array.flatten()
    x = imagen_a_planar / 255.0
    return x

In [48]:
def carga_datos(path): 
    imagenes=[]
    for directorio, subdirectorios, archivos in os.walk(path):
        for archivo in archivos:
            if archivo.endswith('.jpg') or archivo.endswith('.png'):
                archivo_path = os.path.join(directorio, archivo)
                imagenes.append(archivo_path)
    x = []
    ruta=[]
    for i in imagenes:
        ruta.append(i.replace(path,"").split("\\")[2])
        imagen = Image.open(i)
        imagen_rgb = imagen.convert('RGB')
        imagen_rgb = imagen_rgb.resize((348, 348))
        imagen_array = np.array(imagen_rgb)
        imagen_a_planar = imagen_array.flatten()
        imagen_normalizado = imagen_a_planar / 255.0
        x.append(imagen_normalizado)
    data_reshaped = np.array(ruta).reshape(-1, 1) 
    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded = encoder.fit_transform(data_reshaped)
    y = one_hot_encoded.ravel()
    return x,y

In [49]:
def carga_datos_train(path): 
    imagenes=[]
    for directorio, subdirectorios, archivos in os.walk(path):
        for archivo in archivos:
            if archivo.endswith('.jpg') or archivo.endswith('.png'):
                archivo_path = os.path.join(directorio, archivo)
                imagenes.append(archivo_path)
    x = []
    ruta=[]
    for i in imagenes:
        ruta.append(i.replace(path,"").split("\\")[2])
        x.append(imagen(i))
        ruta.append(i.replace(path,"").split("\\")[2])
        x.append(imagen_90(i))
        ruta.append(i.replace(path,"").split("\\")[2])
        x.append(imagen_hor(i))
        ruta.append(i.replace(path,"").split("\\")[2])
        x.append(imagen_ver(i))
        
    data_reshaped = np.array(ruta).reshape(-1, 1) 
    encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoded = encoder.fit_transform(data_reshaped)
    y = one_hot_encoded.ravel()
    return x,y

In [ ]:
X_train= carga_datos_train(path)[0]
y_train= carga_datos_train(path)[1]


In [15]:
X_test= carga_datos(path_test)[0]
y_test= carga_datos(path_test)[1]


c:\Users\jacar\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jacar\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
clf = RandomForestClassifier(max_depth=43, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=43, random_state=0)

In [23]:
y_pred = clf.predict(X_test)

In [30]:


x_train = np.array(X_train)
x_train = x_train.reshape(-1, 348, 348, 3)

In [ ]:
X_test = np.array(X_test)
X_test = X_test.reshape(-1, 348, 348, 3)

In [44]:
y_train.shape

(10560,)

In [25]:

print("Precision:")
print("- Test:", precision_score(y_test, y_pred, average='weighted'))
print("\nRecall:")
print("- Test:", recall_score(y_test, y_pred, average='weighted'))
print("\nF1:")
print("- Test:", f1_score(y_test, y_pred, average='weighted'))
print("\nROC AUC:")
print("- Test:", roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr'))


Precision:
- Test: 0.12977123354436057

Recall:
- Test: 0.19517102615694165

F1:
- Test: 0.1332836134939468

ROC AUC:


c:\Users\jacar\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


- Test: 0.7261908250442778
